# Sinkhorn-Knopp

In [ ]:
import numpy as np
from sinkhorn_knopp import sinkhorn_knopp as skp
import pandas as pd
import torch

sk = skp.SinkhornKnopp()
csv_path = "sim_mtx.csv"
# df = pd.read_csv(csv_path, header=None, sep=',')
# P = torch.tensor(df.values)
# P = np.array([[1,2,3],[4,5,6],[7,8,9]])
P = np.random.rand(100, 100)
P_ds = sk.fit(P)
print(P_ds)
print(np.sum(P_ds, axis=0))
print(np.sum(P_ds, axis=1))

# Biclustering

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.datasets import make_checkerboard
from sklearn.cluster import SpectralBiclustering
from sklearn.metrics import consensus_score


n_clusters = (4, 3)
data, rows, columns = make_checkerboard(
    shape=(300, 300), n_clusters=n_clusters, noise=10, shuffle=False, random_state=0
)

plt.matshow(data, cmap=plt.cm.Blues)
plt.title("Original dataset")

# shuffle clusters
rng = np.random.RandomState(0)
row_idx = rng.permutation(data.shape[0])
col_idx = rng.permutation(data.shape[1])
data = data[row_idx][:, col_idx]

plt.matshow(data, cmap=plt.cm.Blues)
plt.title("Shuffled dataset")

model = SpectralBiclustering(n_clusters=n_clusters, method="log", random_state=0)
model.fit(data)
score = consensus_score(model.biclusters_, (rows[:, row_idx], columns[:, col_idx]))

print("consensus score: {:.1f}".format(score))

fit_data = data[np.argsort(model.row_labels_)]
fit_data = fit_data[:, np.argsort(model.column_labels_)]

plt.matshow(fit_data, cmap=plt.cm.Blues)
plt.title("After biclustering; rearranged to show biclusters")

plt.matshow(
    np.outer(np.sort(model.row_labels_) + 1, np.sort(model.column_labels_) + 1),
    cmap=plt.cm.Blues,
)
plt.title("Checkerboard structure of rearranged data")

plt.show()

In [ ]:
np.outer(np.sort(model.row_labels_) + 1, np.sort(model.column_labels_) + 1),
# np.outer(model.row_labels_ + 1, model.column_labels_ )

# get_clsdst_transform():

In [ ]:
import torchvision.transforms as transforms
from augmentations.aug import get_jigsaw
from prc_functions.dataloader import get_clsdst_transform, get_dataloader
import torchvision.transforms as transforms
import torchvision

transform = transforms.Compose([
    # transforms.Resize(224, interpolation=InterpolationMode.BICUBIC),
    transforms.Resize(224),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    transforms.ToPILImage(),
    transforms.Lambda(get_jigsaw),
])
dataset = torchvision.datasets.CIFAR10(root="/nas/data/syamagami/tta/rmt", train=False, download=True, transform=transform)
im, label = dataset[0]

im

# Shuffle on Tensor Patch

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

def shuffle_image_tensor(image_tensor:torch.Tensor, grid:int=4, C=3) -> torch.Tensor:
    """ テンソル型に変換した後の画像をパッチに分割して Block Shuffle する.
    args:
        image_tensor (torch.Tensor): [B, C, H, W]
        grid (int): 1辺のパッチ数
        C (int): チャンネル数
    return:
        shuffled_image_tensor (torch.Tensor): [B, C, H, W]
    """
    patch_num = grid ** 2
    image_edge = image_tensor.shape[-1]
    
    # 最も近いgridの倍数にリサイズ
    while image_edge % grid != 0:
        image_edge += 1

    h, w = image_edge, image_edge
    patch_h, patch_w = h // grid, w // grid
    patch_i = torch.randperm(patch_num)

    unfold = image_tensor.unfold(2, patch_h, patch_h).unfold(3, patch_h, patch_w)  # [B, C, grid, grid, patch_h, patch_w]
    shuffled = unfold.reshape(-1, C, patch_num, patch_h, patch_w)[:, :, patch_i, :]  # [B, C, patch_num, patch_h, patch_w]
    shuffled_image_tensor = shuffled.reshape(-1, C, grid, grid, patch_h, patch_w).permute(0, 1, 2, 4, 3, 5).reshape(-1, C, h, w)  # [B, C, H, W]

    return shuffled_image_tensor


transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
# 画像を読み込む
image1 = Image.open("prc_functions/image.jpg").resize((32,32))  # 画像のパスを指定してください
image2 = Image.open("prc_functions/image2.jpg").resize((32,32))  # 画像のパスを指定してください
# 画像をテンソルに変換する
image_tensor1 = transform(image1)
image_tensor2 = transform(image2)
image_tensor = torch.cat((image_tensor1.unsqueeze(0), image_tensor2.unsqueeze(0)) , dim=0)  # [B, C, H, W]

bs_transform = transforms.Compose([
    # transforms.Lambda(lambda im: shuffle_image_tensor(im, grid=4)),
    transforms.Lambda(shuffle_image_tensor),
    transforms.Resize((224, 224)),
])
im = bs_transform(image_tensor)
transforms.ToPILImage()(im[1])

# Mine

In [ ]:
import torch
import numpy as np
import random

# シード値を指定する
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

In [ ]:
from models.mine import Mine, MineTrainer

x = np.random.multivariate_normal(mean=[0,0], cov=[[1,0],[0,1]], size = 300)
y = np.random.multivariate_normal(mean=[0,0], cov=[[1,0.8],[0.8,1]], size = 300)

print(x.shape, y.shape)

In [ ]:
mine = MineTrainer()

joint_data = mine.sample_batch(y, sample_mode='joint')
marginal_data = mine.sample_batch(y, sample_mode='marginal')

result_indep = mine.train(x)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

result_indep_ma = mine.ma(result_indep)
print(result_indep_ma[-1])
plt.plot(range(len(result_indep_ma)),result_indep_ma)

# MINE Loss

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

df = pd.read_csv("csv_clustered_mtx.csv", header=None)
mtx = torch.Tensor(df.values)
diag = torch.diag(mtx).long()
features = torch.randn(len(diag), 512).float()

In [ ]:
from models.mine import Mine, MineTrainer
mine_model = MineTrainer()
mean_feat_per_clust = [features[diag == clust].mean(dim=0) for clust in range(torch.max(diag) + 1)]
# mine_loss = 0.
for i in range(len(mean_feat_per_clust)):
    for j in range(i+1, len(mean_feat_per_clust)):
        data = torch.stack([mean_feat_per_clust[i], mean_feat_per_clust[j]], dim=1)
        mine_model.train(data)

In [ ]:
mean_feat_per_clust[0].shape

# My Vision Transformer

In [ ]:
from models.my_transformer import MyTransformer
import torch
transformer_width = 512
transformer_layers = 12
transformer_heads = 8
context_length = 77

transformer = MyTransformer(
    width=transformer_width,
    layers=transformer_layers,
    heads=transformer_heads,
    # context_length=context_length,
    context_length=200,
    attn_mask="build_attention_mask"
)

In [ ]:
x = torch.randn(200, 512)
print(x.shape)
print(x)
transformer(x)